# O Problema
Sliding Puzzle - Bloco Deslizante

In [34]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [35]:
import numpy as np
from collections import deque
import copy

In [36]:
class Puzzle:
    def __init__(self, state=None):
        self.zeroIndex = [1, 0]
        self.initialState = state
        self.goalState = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    def random(self):
        rng = np.random.default_rng()
        ss = np.arange(9).reshape(3, 3)
        rng.shuffle(ss)
        rng.shuffle(ss, 1)
        self.initialState = ss

    def goalTest(self, state):
        return np.array_equal(state, self.goalState)

    def actions(self, state):
        # (l)eft (d)own (u)p (r)ight
        actions = np.array([["dr", "ldr", "ld"],
                            ["dur", "ldur", "ldu"],
                            ["ur", "lur",  "lu"]])
        zeroIndex = np.where(state == 0)
        return actions[zeroIndex][0] # string of legal actions

    def result(self, state, action):
        zeroIndex = np.where(state == 0)
        mov = {
        'l': (0, -1),  # (l)eft
        'd': (1, 0),   # (d)own
        'u': (-1, 0),  # (u)p
        'r': (0, 1)    # (r)ight
        }
        zeroIndexNdarray = np.column_stack(zeroIndex).flatten()
        swapIndex = tuple( zeroIndexNdarray+ mov[action])
        s = copy.deepcopy(state)
        s[zeroIndex], s[swapIndex] = s[swapIndex], s[zeroIndex]
        return s

In [37]:
class Node:
    def __init__(self, state=None, parent=None):
        self.state = state
        self.parent = parent
        self.action = None
        self.child = []

In [38]:
def childNode(problem, parent, action):
    child = Node()
    child.state = problem.result(parent.state, action)
    child.parent = parent
    child.action = action
    return child

In [39]:
def solution(node):
    path = [node]
    while node.parent != None:
        path.append(node.parent)
        node = node.parent
    path.reverse()
    return path

In [40]:
def solve(problem, search):
    if search == "bfs":
        return breadthFirstSearch(problem)
    if search == "dps":
        return depthFirstSearch(problem)

In [41]:
solvableState = np.array([[1, 8, 2], [0, 4, 3], [7, 6, 5]])
puzzle = Puzzle(solvableState)

## Busca em largura

In [42]:
def breadthFirstSearch(problem):
    node = Node(puzzle.initialState)
    if problem.goalTest(node.state):
        return solution(node)
    frontier = deque([node])
    explored = set()
    while True:
        if not bool(frontier):
            return "Failure"
        node = frontier.popleft()
        explored.add(tuple(tuple(row) for row in node.state))
        for action in problem.actions(node.state):
            child = childNode(problem, node, action)
            tupleChildState = tuple(tuple(row) for row in child.state)
            if (tupleChildState not in explored) or (tupleChildState not in frontier):
                if problem.goalTest(child.state):
                    return solution(child)
                frontier.append(child)

In [43]:
solution = solve(puzzle, "bfs")
for node in solution:
    print(node.state)
    print()

[[1 8 2]
 [0 4 3]
 [7 6 5]]

[[1 8 2]
 [4 0 3]
 [7 6 5]]

[[1 0 2]
 [4 8 3]
 [7 6 5]]

[[1 2 0]
 [4 8 3]
 [7 6 5]]

[[1 2 3]
 [4 8 0]
 [7 6 5]]

[[1 2 3]
 [4 8 5]
 [7 6 0]]

[[1 2 3]
 [4 8 5]
 [7 0 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]



## Busca em Profundidade

In [44]:
def depthFirstSearch(problem):
    return 0

## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

